In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/1-revenue-all-df/df_1.pkl
/kaggle/input/1-revenue-all-df/__results__.html
/kaggle/input/1-revenue-all-df/df_5.pkl
/kaggle/input/1-revenue-all-df/df_2.pkl
/kaggle/input/1-revenue-all-df/df_6.pkl
/kaggle/input/1-revenue-all-df/df_3.pkl
/kaggle/input/1-revenue-all-df/__notebook__.ipynb
/kaggle/input/1-revenue-all-df/df_7.pkl
/kaggle/input/1-revenue-all-df/custom.css
/kaggle/input/1-revenue-all-df/df_4.pkl
/kaggle/input/1-revenue-all-df/__output__.json
/kaggle/input/1-revenue-all-df/__results___files/__results___13_0.png


In [2]:
import pandas as pd
import numpy as np
import gc; gc.enable()

import os, sys, gc, time, warnings, pickle, random, psutil
from scipy import sparse
from  datetime import datetime, timedelta

import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.ensemble import RandomForestClassifier,BaggingRegressor
from sklearn.model_selection import cross_val_score, cross_validate,cross_val_predict
from sklearn.model_selection import  train_test_split,KFold,StratifiedKFold,TimeSeriesSplit

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, make_union

from sklearn.preprocessing import LabelEncoder,MinMaxScaler, RobustScaler
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer

import category_encoders as ce

from category_encoders import WOEEncoder, TargetEncoder
from sklearn.impute import SimpleImputer

from sklearn.linear_model import SGDRegressor
from dask_ml.wrappers import Incremental




%matplotlib inline
warnings.filterwarnings("ignore")

In [3]:
import psutil
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2)

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    
    gc.collect()
    return df

In [4]:
get_memory_usage()

0.21

In [5]:
# df_20 = pd.read_pickle('/kaggle/input/part-1-new/df_20.pkl')
# df_200 = pd.read_pickle('/kaggle/input/part-1-new/df_200.pkl')
# df_ca_1 = pd.read_pickle('/kaggle/input/part-1-new/df_ca_1.pkl')
df = pd.read_pickle('/kaggle/input/1-revenue-all-df/df_1.pkl')

In [6]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,wday,...,snap_CA,snap_TX,snap_WI,event_name_1,event_type_1,event_name_2,event_type_2,day,week,sell_price
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,HOBBIES_1_141_CA_1_evaluation,HOBBIES_1_141,HOBBIES_1,HOBBIES,CA_1,CA,1,2,11101,1,...,0,0,0,0,0,0,0,29,4,14.26
2011-01-29,FOODS_3_587_WI_2_evaluation,FOODS_3_587,FOODS_3,FOODS,WI_2,WI,1,0,11101,1,...,0,0,0,0,0,0,0,29,4,2.48
2011-01-29,HOBBIES_1_345_WI_1_evaluation,HOBBIES_1_345,HOBBIES_1,HOBBIES,WI_1,WI,1,0,11101,1,...,0,0,0,0,0,0,0,29,4,7.44
2011-01-29,FOODS_3_263_TX_1_evaluation,FOODS_3_263,FOODS_3,FOODS,TX_1,TX,1,4,11101,1,...,0,0,0,0,0,0,0,29,4,2.68
2011-01-29,HOBBIES_1_297_CA_3_evaluation,HOBBIES_1_297,HOBBIES_1,HOBBIES,CA_3,CA,1,2,11101,1,...,0,0,0,0,0,0,0,29,4,13.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-05-22,FOODS_3_377_TX_2_evaluation,FOODS_3_377,FOODS_3,FOODS,TX_2,TX,1941,18,11617,2,...,0,0,0,0,0,0,0,22,20,1.68
2016-05-22,FOODS_3_389_TX_2_evaluation,FOODS_3_389,FOODS_3,FOODS,TX_2,TX,1941,27,11617,2,...,0,0,0,0,0,0,0,22,20,1.68
2016-05-22,HOUSEHOLD_1_029_CA_3_evaluation,HOUSEHOLD_1_029,HOUSEHOLD_1,HOUSEHOLD,CA_3,CA,1941,2,11617,2,...,0,0,0,0,0,0,0,22,20,7.48


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 852648 entries, 2011-01-29 to 2016-05-22
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   id            852648 non-null  category
 1   item_id       852648 non-null  category
 2   dept_id       852648 non-null  category
 3   cat_id        852648 non-null  category
 4   store_id      852648 non-null  category
 5   state_id      852648 non-null  category
 6   d             852648 non-null  int16   
 7   sales         852648 non-null  int16   
 8   wm_yr_wk      852648 non-null  int16   
 9   wday          852648 non-null  int8    
 10  month         852648 non-null  int8    
 11  year          852648 non-null  int16   
 12  snap_CA       852648 non-null  int8    
 13  snap_TX       852648 non-null  int8    
 14  snap_WI       852648 non-null  int8    
 15  event_name_1  852648 non-null  int8    
 16  event_type_1  852648 non-null  int8    
 17  event_name_2 

In [8]:
gc.collect()

24

In [9]:
get_memory_usage()

0.28

In [10]:
gc.collect()

40

## 2. Feature Engineering
### 2.1. Lag Features

In [11]:
def lag_features(df):
    '''
    Function returns data frame with lag features.
    
    '''
    #create data frame from data frame for lag features
    lag_df = df[['id','item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd', 'sales']]
    lag_group = lag_df.groupby(['id','item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])
    
    # cannot use .pct_change() because of some 0 values, which give a lot of 'inf' and 'NaN'
    # create lag(shift) features: this represents % cahnge of sales 'shift' days ago
    shifts = [7,8,9,10,11,12,13,14,28,35,42]
    lag_diff = lag_group['sales'].transform(lambda x: x.diff().astype(np.float32))
    lag_max = lag_group['sales'].transform(lambda x: x.mean().astype(np.float32))
    lag_df['pct_norm'] = (lag_diff/lag_max).astype(np.float32)
    
    for i in shifts:
        lag_df[f'pct_shift_{i}'] = lag_group['pct_norm'].shift(i)
    
    
    # use shift min=7 with windows for pct change of rolling mean and std dev. calc  
    windows = [7,14,28]
    shifts = [7, 14] 
    for i in shifts:
        
        for j in windows:
            lag_df[f'pct_rmean{j}_lag{i}'] = \
            lag_group[f'pct_shift_{i}'].transform(lambda x: x.rolling(j).mean().astype(np.float32))
            
            lag_df[f'pct_rstd{j}_lag{i}'] = \
            lag_group[f'pct_shift_{i}'].transform(lambda x: x.rolling(j).std().astype(np.float32))

            
    del lag_df['pct_norm']
    gc.collect()
    
    return lag_df


def merge_lag_df(df,lag_df):
    
    '''
    Function returns merged df with lag features df.
    '''
    ### merge lag features with data frame
    
    lag_df.drop(columns = 'sales',axis=1,inplace=True)
    df = df.merge(lag_df, 
                  on=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd'], 
                  how='left', left_index=True)
    gc.collect()

    return df

def remove_nan(df):
    '''
    Function drops NaN, created by lag features.
    '''
    #df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna()
    gc.collect()
    return df

SHIFT = 7

### 2.2. Price Features
def price_features(df):
    
    '''
    Function returns data frame with price features(moving average = 14 days).
    '''
#     price_group = df['sell_price'].pct_change().rolling(14)
#     df['pct_price_mean']= price_group.mean().shift(SHIFT)
#     df['pct_price_max']= price_group.max().shift(SHIFT)
#     df['pct_price_min']= price_group.min().shift(SHIFT)
#     df['pct_price_std']= price_group.std().shift(SHIFT)
    
    price_group = df.groupby(['id','item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])

    df['pct_price'] = price_group['sell_price'].pct_change().astype(np.float32)
    
    df['pct_price_mean'] = price_group['pct_price'].transform(lambda x: x.rolling(14).mean().astype(np.float32))
    df['pct_price_mean'] = price_group['pct_price_mean'].shift(SHIFT)
    
    gc.collect()
    
    df['pct_price_std'] = price_group['pct_price'].transform(lambda x: x.rolling(14).std().astype(np.float32))
    df['pct_price_std'] = price_group['pct_price_std'].shift(SHIFT)
    
    df['pct_price_max'] = price_group['pct_price'].transform(lambda x: x.rolling(14).max().astype(np.float32))
    df['pct_price_max'] = price_group['pct_price_max'].shift(SHIFT)
    
    df['pct_price_min'] = price_group['pct_price'].transform(lambda x: x.rolling(14).min().astype(np.float32))
    df['pct_price_min'] = price_group['pct_price_min'].shift(SHIFT)
    
    del df['pct_price']
    gc.collect()
    
    return df


## 3. Baseline Model
## 3.1. LIGHT GBM

In [12]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [13]:
def predict_lgb(X_train, X_test, y_train, y_test):
    
    '''
    Function prints RMSE for train and test data.
    '''
    start_time = time.time()
    
    # Create X, Y
    # 'wday' duplicates 'weekday'- put in remove_col
#     remove_cols = ['sales', 'revenue'] 
    
#     cat_cols = ['id','item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
#     num_cols = [c for c in df.columns.tolist() if c not in (remove_cols+cat_cols)]
#     used_col = cat_cols+num_cols
    
    
#     y = df['sales']
#     X = df[used_col]
#     del df
#     gc.collect()
    
    
#     # Split train, test(last 7 days) # new
#     y_train, y_test = y[:'2016-05-15'], y['2016-05-16':]
#     del y
#     gc.collect()

#     X_train, X_test = X[:'2016-05-15'], X['2016-05-16':]
#     del X
#     gc.collect()
    
    ### Make numerical pipe
    imputer = SimpleImputer()
    scaler = RobustScaler()
    pca = PCA(n_components=0.9, random_state = 42)
    num_pipe = make_pipeline(imputer,scaler,pca)
    
    ### Make categorical pipe
    #encoder = ce.TargetEncoder(cols=cat_cols, handle_missing="value")
    encoder = ce.cat_boost.CatBoostEncoder(cols=cat_cols, handle_missing="value")
    cat_pipe = make_pipeline(encoder)
    
    
    # Define preprocessor
    preprocessor = ColumnTransformer(
        transformers=[('num', num_pipe, num_cols),
                      ('cat', cat_pipe, cat_cols)])
   
    # Define model
    model = lgb.LGBMRegressor(objective="poisson", 
                              metric="rmse",
                              force_row_wise = True,
                              learning_rate = 0.02, #0.04,
                              sub_row = 0.75,
                              bagging_freq = 1,
                              lambda_l2 = 0.1,
                              verbosity = 1,
                              num_iterations = 100,
                              num_leaves = 20,#40,
                              max_depth = 5, #20,
                              min_data_in_leaf = 10)#11)
    
    # Make main pipeline
    pipe = make_pipeline(preprocessor, model)

    ### cross_validate
    
#     block = BlockingTimeSeriesSplit(n_splits=5)
    #ts_cv = TimeSeriesSplit(n_splits=10)
    #kfcv = KFold(n_splits=5)

#     scores = cross_validate(pipe, X_train, y_train, cv=block, 
#                              scoring='neg_root_mean_squared_error',
#                              n_jobs=-1,return_train_score=True)
    #sorted(scores.keys()) ['fit_time', 'score_time', 'test_score', 'train_score']

#     print('Train scores:', scores['train_score'])
#     print('Test scores:', scores['test_score'])

#     print('Train mean:',scores['train_score'].mean(),"+/-",scores['train_score'].std())
#     print('Test mean:',scores['test_score'].mean(),"+/-",scores['test_score'].std())
    
#     del scores
#     gc.collect()
    
    #### fit and predict

    pipe.fit(X_train,y_train)
    y_pred_train=pipe.predict(X_train)
    y_pred=pipe.predict(X_test)

    print('--------------------')
    print("RMSE_train:",(mean_squared_error(y_train.values, y_pred_train))**0.5)
    print("RMSE_test:",(mean_squared_error(y_test.values, y_pred))**0.5)
    print('--------------------')
    print()
    print('Naive benchmarks:')
    print('Predicted Test week average sales:',y_pred.mean())
    print('True Average Sales at:')
    print('Test week:',y_test.values.mean())
#     print('Week before test:', y['2016-04-10':'2016-04-16'].values.mean())
#     print('Last year at test week:', y['2015-04-20':'2015-04-26'].values.mean())
    print('--------------------')
    
    end_time = time.time()
    print('time:', (end_time-start_time)/60, 'min')

    
#     print('Sales')

#     print('Predicted Test week:',np.round(y_pred.tolist()))
#     print('True Test week:',y_test.values.tolist())
#     print('Week before test:', y['2016-04-11':'2016-04-17'].values.tolist())
#     print('Last year @ test week:', y['2015-04-20':'2015-04-26'].values.tolist())
    
    
    del y_pred_train
    gc.collect()

    
    return y_pred

## 4. Vanilla Linear Regression.

In [14]:
def v_lin_reg(X_train, X_test, y_train, y_test):
    
    '''
    Vanilla linear reg model
    Function prints RMSE for train and test data.
    '''
    start_time = time.time()
    
    ### Make numerical pipe
    imputer = SimpleImputer()
    scaler = RobustScaler()
    pca = PCA(n_components=0.9, random_state = 42)
    num_pipe = make_pipeline(imputer,scaler,pca)
    
    ### Make categorical pipe
    #encoder = ce.TargetEncoder(cols=cat_cols, handle_missing="value")
    encoder = ce.cat_boost.CatBoostEncoder(cols=cat_cols, handle_missing="value")
    cat_pipe = make_pipeline(encoder)
    
    
    # Define preprocessor
    preprocessor = ColumnTransformer(
        transformers=[('num', num_pipe, num_cols),
                      ('cat', cat_pipe, cat_cols)])
   
    # Define model
    model = LinearRegression(n_jobs=-1)
    
    # Make main pipeline
    pipe = make_pipeline(preprocessor, model)

    ### cross_validate
    
    #block = BlockingTimeSeriesSplit(n_splits=5)
    ##ts_cv = TimeSeriesSplit(n_splits=10)
    ##kfcv = KFold(n_splits=5)

#     scores = cross_validate(pipe, X_train, y_train, cv=block, 
#                              scoring='neg_root_mean_squared_error',
#                              return_train_score=True) #n_jobs=-1,
    ###sorted(scores.keys()) ['fit_time', 'score_time', 'test_score', 'train_score']

#     print('Train scores:', scores['train_score'])
#     print('Test scores:', scores['test_score'])

#     print('Train mean:',scores['train_score'].mean(),"+/-",scores['train_score'].std())
#     print('Test mean:',scores['test_score'].mean(),"+/-",scores['test_score'].std())
    
#     del scores
#     gc.collect()
    
    #### fit and predict

    pipe.fit(X_train,y_train)
    y_pred_train=pipe.predict(X_train)
    y_pred=pipe.predict(X_test)

    print('--------------------')
    print("RMSE_train:",(mean_squared_error(y_train.values, y_pred_train))**0.5)
    print("RMSE_test:",(mean_squared_error(y_test.values, y_pred))**0.5)
    print('--------------------')
    print()
    print('Naive benchmarks:')
    print('Predicted Test week average sales:',y_pred.mean())
    print('True Average Sales at:')
    print('Test week:',y_test.values.mean())
#     print('Week before test:', y['2016-04-10':'2016-04-16'].values.mean())
#     print('Last year at test week:', y['2015-04-20':'2015-04-26'].values.mean())
    print('--------------------')
    
    end_time = time.time()
    print('time:', (end_time-start_time)/60, 'min')
    
    del y_pred_train
    gc.collect()
    
    return y_pred

In [15]:

# est = SGDClassifier(loss='log', penalty='l2', tol=1e-3)
# inc = Incremental(est, scoring='accuracy')
# inc.fit(X_train, y_train, classes=classes)
# inc.score(X_test, y_test)
# inc.predict(X_test)  # Predict produces lazy dask arrays
# inc.predict(X_test)[:100].compute()  # call compute to get results
# inc.score(X_test, y_test)



def sgd(X_train, X_test, y_train, y_test):
    
    '''
    Function prints RMSE for train and test data.
    '''
    start_time = time.time()
    
    
    ### Make numerical pipe
    imputer = SimpleImputer()
    scaler = RobustScaler()
    pca = PCA(n_components=0.9, random_state = 42)
    num_pipe = make_pipeline(imputer,scaler,pca)
    
    ### Make categorical pipe
    #encoder = ce.TargetEncoder(cols=cat_cols, handle_missing="value")
    encoder = ce.cat_boost.CatBoostEncoder(cols=cat_cols, handle_missing="value")
    cat_pipe = make_pipeline(encoder)
    
    
    # Define preprocessor
    preprocessor = ColumnTransformer(
        transformers=[('num', num_pipe, num_cols),
                      ('cat', cat_pipe, cat_cols)])
   
    # Define model
#     model = LinearRegression()
    model = SGDRegressor(shuffle=False, 
                         penalty='elasticnet',
                         loss='epsilon_insensitive',
                         learning_rate='adaptive')
    
    #inc = Incremental(model) #, scoring='accuracy')


    # Make main pipeline
    pipe = make_pipeline(preprocessor, model)

    ### cross_validate
    
    block = BlockingTimeSeriesSplit(n_splits=5)
    #ts_cv = TimeSeriesSplit(n_splits=10)
    #kfcv = KFold(n_splits=5)
    
    
    
    scores = cross_validate(pipe, X_train, y_train, cv=block, 
                             scoring='neg_root_mean_squared_error',
                             return_train_score=True) #n_jobs=-1,
    #sorted(scores.keys()) ['fit_time', 'score_time', 'test_score', 'train_score']

    print('Train scores:', scores['train_score'])
    print('Test scores:', scores['test_score'])
    
    print('Train mean:',scores['train_score'].mean(),"+/-",scores['train_score'].std())
    print('Test mean:',scores['test_score'].mean(),"+/-",scores['test_score'].std())
    
    del scores
    gc.collect()
    
    
    #### fit and predict
    pipe.fit(X_train,y_train)
    
    y_pred_train=pipe.predict(X_train)
    y_pred=pipe.predict(X_test)

    print('--------------------')
    print("RMSE_train:",(mean_squared_error(y_train.values, y_pred_train))**0.5)
    print("RMSE_test:",(mean_squared_error(y_test.values, y_pred))**0.5)
    print('--------------------')
    print()
    print('Naive benchmarks:')
    print('Predicted Test week average sales:',y_pred.mean())
    print('True Average Sales at:')
    print('Test week:',y_test.values.mean())
#     print('Week before test:', y['2016-04-10':'2016-04-16'].values.mean())
#     print('Last year at test week:', y['2015-04-20':'2015-04-26'].values.mean())
    print('--------------------')
    
    end_time = time.time()
    print('time:', (end_time-start_time)/60, 'min')
    
    del y_pred_train
    gc.collect()
    
    return y_pred

1. ### test vanilla linear reg on df_1

In [16]:
### train: days [1; 1913] or [2011-01-29; 2016-04-24]

### test week_1: days [1914 : 1920] or [2016-04-25 : 2016-05-01]
df = df.loc[(df.d<=1913)| ((df.d>=1914)&(df.d<=1920))]
#df.loc[(df.d<=1913)| ((df.d>=1914)&(df.d<=1920))][:'2016-04-24'] # train split
#df.loc[(df.d<=1913)| ((df.d>=1914)&(df.d<=1920))]['2016-04-25':] # test split


### test week_2: days [1921 : 1927] or [2016-05-02 : 2016-05-08]
# df = df.loc[(df.d<=1913)| ((df.d>=1921)&(df.d<=1927))]
# #df.loc[(df.d<=1913)| ((df.d>=1921)&(df.d<=1927))][:'2016-04-24'] # train split
# #df.loc[(df.d<=1913)| ((df.d>=1921)&(df.d<=1927))]['2016-04-25':] # test split


### test week_3: days [1928 : 1934] or [2016-05-09 : 2016-05-15]
# df = df.loc[(df.d<=1913)| ((df.d>=1928)&(df.d<=1934))]
# #df.loc[(df.d<=1913)| ((df.d>=1928)&(df.d<=1934))][:'2016-04-24'] # train split
# #df.loc[(df.d<=1913)| ((df.d>=1928)&(df.d<=1934))]['2016-04-25':] # test split

### test week_4: days [1935 : 1941] or [2016-05-16 : 2016-05-22]
# df = df.loc[(df.d<=1913)| ((df.d>=1935)&(df.d<=1941))]
# #df.loc[(df.d<=1913)| ((df.d>=1935)&(df.d<=1941))][:'2016-04-24'] # train split
# #df.loc[(df.d<=1913)| ((df.d>=1935)&(df.d<=1941))]['2016-04-25':] # test split

In [17]:
### create features: 
start_time = time.time()

lag = lag_features(df)
df = merge_lag_df(df, lag)
del lag
df = price_features(df)
df = remove_nan(df)

end_time = time.time()
print('time:', (end_time-start_time)/60, 'min')

time: 0.27414055665334064 min


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 819304 entries, 2011-03-13 to 2016-05-01
Data columns (total 49 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   id                 819304 non-null  category
 1   item_id            819304 non-null  category
 2   dept_id            819304 non-null  category
 3   cat_id             819304 non-null  category
 4   store_id           819304 non-null  category
 5   state_id           819304 non-null  category
 6   d                  819304 non-null  int16   
 7   sales              819304 non-null  int16   
 8   wm_yr_wk           819304 non-null  int16   
 9   wday               819304 non-null  int8    
 10  month              819304 non-null  int8    
 11  year               819304 non-null  int16   
 12  snap_CA            819304 non-null  int8    
 13  snap_TX            819304 non-null  int8    
 14  snap_WI            819304 non-null  int8    
 15  event_name_1      

In [19]:
reduce_mem_usage(df)

Mem. usage decreased to 72.69 Mb (37.6% reduction)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,wday,...,pct_rmean7_lag14,pct_rstd7_lag14,pct_rmean14_lag14,pct_rstd14_lag14,pct_rmean28_lag14,pct_rstd28_lag14,pct_price_mean,pct_price_std,pct_price_max,pct_price_min
date,,,,,,,,,,,,,,,,,,,,,
2011-03-13,FOODS_2_285_WI_2_evaluation,FOODS_2_285,FOODS_2,FOODS,WI_2,WI,44,4,11107,2,...,-0.000000,0.479248,0.040680,0.491211,0.015251,0.382324,0.000000,0.00000,0.000000,0.0
2011-03-13,FOODS_2_368_WI_2_evaluation,FOODS_2_368,FOODS_2,FOODS,WI_2,WI,44,11,11107,2,...,-0.000000,1.176758,-0.294922,2.830078,0.016388,3.205078,0.000000,0.00000,0.000000,0.0
2011-03-13,FOODS_3_572_CA_3_evaluation,FOODS_3_572,FOODS_3,FOODS,CA_3,CA,44,10,11107,2,...,-0.081848,0.466553,-0.040924,0.540039,-0.025574,0.492920,0.000000,0.00000,0.000000,0.0
2011-03-13,FOODS_3_389_CA_3_evaluation,FOODS_3_389,FOODS_3,FOODS,CA_3,CA,44,19,11107,2,...,0.039124,0.437744,-0.007820,0.530762,0.017609,0.471924,0.000000,0.00000,0.000000,0.0
2011-03-13,FOODS_3_458_CA_3_evaluation,FOODS_3_458,FOODS_3,FOODS,CA_3,CA,44,29,11107,2,...,-0.044952,0.427979,0.028091,0.378662,-0.002810,0.410889,0.001553,0.00581,0.021744,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-05-01,HOBBIES_1_155_CA_1_evaluation,HOBBIES_1_155,HOBBIES_1,HOBBIES,CA_1,CA,1920,2,11614,2,...,-0.101807,1.247070,0.025452,1.122070,-0.012726,1.038086,0.000000,0.00000,0.000000,0.0
2016-05-01,FOODS_3_586_WI_2_evaluation,FOODS_3_586,FOODS_3,FOODS,WI_2,WI,1920,13,11614,2,...,0.038239,0.389893,0.019119,0.390625,0.016724,0.449219,0.000000,0.00000,0.000000,0.0
2016-05-01,HOBBIES_1_147_CA_1_evaluation,HOBBIES_1_147,HOBBIES_1,HOBBIES,CA_1,CA,1920,8,11614,2,...,-0.152466,0.814453,-0.060974,0.787109,0.007622,1.285156,0.000000,0.00000,0.000000,0.0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 819304 entries, 2011-03-13 to 2016-05-01
Data columns (total 49 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   id                 819304 non-null  category
 1   item_id            819304 non-null  category
 2   dept_id            819304 non-null  category
 3   cat_id             819304 non-null  category
 4   store_id           819304 non-null  category
 5   state_id           819304 non-null  category
 6   d                  819304 non-null  int16   
 7   sales              819304 non-null  int16   
 8   wm_yr_wk           819304 non-null  int16   
 9   wday               819304 non-null  int8    
 10  month              819304 non-null  int8    
 11  year               819304 non-null  int16   
 12  snap_CA            819304 non-null  int8    
 13  snap_TX            819304 non-null  int8    
 14  snap_WI            819304 non-null  int8    
 15  event_name_1      

In [21]:

# split using only pandas

remove_cols = ['sales', 'revenue'] 
cat_cols = ['id','item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
num_cols = [c for c in df.columns.tolist() if c not in (remove_cols+cat_cols)]
used_col = cat_cols+num_cols
    
    
y = df['sales']
X = df[used_col]
del df
gc.collect()

# Split train, test(last 7 days) # new
y_train, y_test = y[:'2016-04-24'], y['2016-04-25':]
del y
gc.collect()

X_train, X_test = X[:'2016-04-24'], X['2016-04-25':]
del X
gc.collect()

0

In [22]:
#Predict using lgb model
# y_pred = v_lin_reg(X_train, X_test, y_train, y_test)
y_pred = predict_lgb(X_train, X_test, y_train, y_test)

--------------------
RMSE_train: 13.412612449045945
RMSE_test: 9.466055189574256
--------------------

Naive benchmarks:
Predicted Test week average sales: 11.593260550828024
True Average Sales at:
Test week: 12.5379764189745
--------------------
time: 3.138801662127177 min


In [23]:
result = pd.DataFrame(y_pred)
result.to_pickle('wk_1_pred_1.pkl')

In [24]:
# len(y_pred)

In [25]:
#Predict using SGD
#y_pred = sgd(X_train, X_test, y_train, y_test)

In [26]:
#Predict using lgb
#y_pred = predict_lgb(X_train, X_test, y_train, y_test)